In [18]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re

In [19]:
df = pd.read_csv("train_E6oV3lV.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [20]:
hate_tweet = df[df.label == 1]
hate_tweet.head()

,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...
14,15,1,no comment! in #australia #opkillingbay #se...
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'...


In [21]:
normal_tweet = df[df.label == 0]
normal_tweet.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [22]:
# Hate Word clouds
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

text = " ".join(review for review in hate_tweet.tweet)
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
fig = plt.figure(figsize = (20, 6)) 
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

ModuleNotFoundError: No module named 'wordcloud'

In [23]:
#distributions
df_Stat=df[['label','tweet']].groupby('label').count().reset_index()
df_Stat.columns=['label','count']
df_Stat['percentage']=(df_Stat['count']/df_Stat['count'].sum())*100
df_Stat

,label,count,percentage
0,0,29720,92.98542
1,1,2242,7.01458


In [24]:
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

In [25]:
df['processed_tweets'] = df['tweet'].apply(process_tweet)
df.head()

,id,label,tweet,processed_tweets
0,1,0,@user when a father is dysfunctional and is s...,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit i can t use cause they ...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,factsguide society now motivation


In [26]:
#As this dataset is highly imbalance we have to balance this by over sampling
cnt_non_fraud = df[df['label'] == 0]['processed_tweets'].count()
df_class_fraud = df[df['label'] == 1]
df_class_nonfraud = df[df['label'] == 0]
df_class_fraud_oversample = df_class_fraud.sample(cnt_non_fraud, replace=True)
df_oversampled = pd.concat([df_class_nonfraud, df_class_fraud_oversample], axis=0)

print('Random over-sampling:')
print(df_oversampled['label'].value_counts())

Random over-sampling:
0    29720
1    29720
Name: label, dtype: int64


In [27]:
# Split data into training and test sets
from sklearn.model_selection import train_test_split
X = df_oversampled['processed_tweets']
y = df_oversampled['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = None)

In [28]:
X_train[0]

'when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run'

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english'  )
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [13]:
count_vect
#why count vectorizer before tfidf
#https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.ZCayyvZBxEY

CountVectorizer(stop_words='english')

In [15]:
x_train_counts = count_vect.fit_transform(X_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [17]:
x_train_counts

<47552x33926 sparse matrix of type '<class 'numpy.int64'>'
	with 345768 stored elements in Compressed Sparse Row format>

In [63]:
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(47552, 34163)
(47552, 34163)


In [64]:
x_test_counts = count_vect.transform(X_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [65]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(n_estimators=500)

In [66]:
predict_rf = model.predict(x_test_tfidf)

In [67]:
print(confusion_matrix(y_test,predict_rf))
print(classification_report(y_test, predict_rf))

[[5667  204]
 [   0 6017]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      5871
           1       0.97      1.00      0.98      6017

    accuracy                           0.98     11888
   macro avg       0.98      0.98      0.98     11888
weighted avg       0.98      0.98      0.98     11888



In [68]:
#Building XGBoost Model
from xgboost import XGBClassifier

model_bow = XGBClassifier(random_state=22,learning_rate=0.9)
model_bow.fit(x_train_tfidf,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.9, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=22,
              reg_alpha=0, reg_lambda=1, ...)

In [69]:
predict_xgb = model_bow.predict(x_test_tfidf)

In [70]:
print(confusion_matrix(y_test,predict_xgb))
print(classification_report(y_test,predict_xgb))

[[5551  320]
 [ 122 5895]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96      5871
           1       0.95      0.98      0.96      6017

    accuracy                           0.96     11888
   macro avg       0.96      0.96      0.96     11888
weighted avg       0.96      0.96      0.96     11888



In [71]:
#SVM Model
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(x_train_tfidf,y_train)

LinearSVC()

In [72]:
predict_svm = lin_clf.predict(x_test_tfidf)

In [73]:
from sklearn.metrics import confusion_matrix,f1_score

print(confusion_matrix(y_test,predict_svm))
print(classification_report(y_test, predict_svm))

[[5725  146]
 [  32 5985]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98      5871
           1       0.98      0.99      0.99      6017

    accuracy                           0.99     11888
   macro avg       0.99      0.98      0.99     11888
weighted avg       0.99      0.99      0.99     11888



In [74]:
#Logistic Regression Model
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42)

#Building Logistic Regression  Model
logreg.fit(x_train_tfidf,y_train)

LogisticRegression(random_state=42)

In [75]:
predict_log = logreg.predict(x_test_tfidf)

print(confusion_matrix(y_test,predict_log))
print(classification_report(y_test, predict_log))

[[5617  254]
 [  62 5955]]
              precision    recall  f1-score   support

           0       0.99      0.96      0.97      5871
           1       0.96      0.99      0.97      6017

    accuracy                           0.97     11888
   macro avg       0.97      0.97      0.97     11888
weighted avg       0.97      0.97      0.97     11888



In [76]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB.fit(x_train_tfidf, y_train)

MultinomialNB()

In [77]:
predict_nb = NB.predict(x_test_tfidf)

print(confusion_matrix(y_test,predict_nb))
print(classification_report(y_test, predict_nb))

[[5389  482]
 [ 107 5910]]
              precision    recall  f1-score   support

           0       0.98      0.92      0.95      5871
           1       0.92      0.98      0.95      6017

    accuracy                           0.95     11888
   macro avg       0.95      0.95      0.95     11888
weighted avg       0.95      0.95      0.95     11888



In [78]:
#Test Data Set
df_test = pd.read_csv("test_tweets_anuFYb8.csv")
df_test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [79]:
df_test.shape

(17197, 2)

In [80]:
df_test['processed_tweets'] = df_test['tweet'].apply(process_tweet)
df_test.head()

,id,tweet,processed_tweets
0,31963,#studiolife #aislife #requires #passion #dedic...,studiolife aislife requires passion dedication...
1,31964,@user #white #supremacists want everyone to s...,white supremacists want everyone to see the ne...
2,31965,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your acne altwaystoheal heal...
3,31966,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",3rd bihday to my amazing hilarious nephew eli ...


In [81]:
X = df_test['processed_tweets']
x_test_counts = count_vect.transform(X)
x_test_tfidf = transformer.transform(x_test_counts)

In [82]:
df_test['predict_nb'] = NB.predict(x_test_tfidf)
df_test[df_test['predict_nb']==1]

,id,tweet,processed_tweets,predict_nb
14,31977,i'll stand behind this #guncontrolplease #se...,i ll stand behind this guncontrolplease sensel...,1
19,31982,thought factory: bbc neutrality on right wing ...,thought factory bbc neutrality on right wing f...,1
26,31989,chick gets fucked hottest naked lady,chick gets fucked hottest naked lady,1
33,31996,suppo the #taiji fisherman! no bullying! no ra...,suppo the taiji fisherman no bullying no racis...,1
34,31997,i say we because i'm speaking collectively. i'...,i say we because i m speaking collectively i v...,1
...,...,...,...,...
17167,49130,people do anything for fucking attention nowad...,people do anything for fucking attention nowadays,1
17176,49139,@user @user are the most racist pay ever!!!!!,are the most racist pay ever,1
17188,49151,"black professor demonizes, proposes nazi style...",black professor demonizes proposes nazi style ...,1
17192,49155,thought factory: left-right polarisation! #tru...,thought factory left right polarisation trump ...,1


In [83]:
df_test['predict_svm'] = lin_clf.predict(x_test_tfidf)
df_test['predict_rf'] = model.predict(x_test_tfidf)
df_test['predict_xgb'] = model_bow.predict(x_test_tfidf)
df_test['predict_logreg'] = logreg.predict(x_test_tfidf)
#df_test.head()

In [84]:
file_name = 'test_predictions_Twitter Hate Analysis.csv'
df_test.to_csv(file_name,index=False)

In [85]:
df = pd.read_csv("test_predictions_Twitter Hate Analysis.csv")
#df.head()

In [86]:
F1 = pd.DataFrame()
F1['Algorithms'] = []
F1['f1 score'] = [f1_score(y_test, predict_rf),f1_score(y_test, predict_svm),f1_score(y_test, predict_xgb),f1_score(y_test, predict_nb),f1_score(y_test, predict_log)]
F1

,Algorithms,f1 score
0,Random Forest,0.983331
1,SVM,0.985347
2,XG Boost,0.963865
3,Navie Bayes,0.952534
4,Logistic Regression,0.974153
